In [15]:

# Import libraries
import pandas as pd
import sqlite3
import os

# Establish connection to SQLite database
connection = sqlite3.connect("employee_database.db")
cursor = connection.cursor()

# Directory path
directory = '/Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge'

# Verify if directory exists and list its contents
if os.path.exists(directory):
    print("Directory exists. Files in the directory:")
    print(os.listdir(directory))
else:
    print("Directory does not exist. Please check the path.")

# Step 1: Create Table Schemas
create_tables_sql = [
    """
    CREATE TABLE IF NOT EXISTS departments (
        dept_no CHAR(4) PRIMARY KEY,
        dept_name VARCHAR(40) NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS employees (
        emp_no INT PRIMARY KEY,
        birth_date DATE NOT NULL,
        first_name VARCHAR(50) NOT NULL,
        last_name VARCHAR(50) NOT NULL,
        hire_date DATE NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS salaries (
        emp_no INT PRIMARY KEY,
        salary INT NOT NULL,
        FOREIGN KEY (emp_no) REFERENCES employees(emp_no)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS titles (
        emp_no INT PRIMARY KEY,
        title VARCHAR(50) NOT NULL,
        FOREIGN KEY (emp_no) REFERENCES employees(emp_no)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dept_emp (
        emp_no INT NOT NULL,
        dept_no CHAR(4) NOT NULL,
        PRIMARY KEY (emp_no, dept_no),
        FOREIGN KEY (emp_no) REFERENCES employees(emp_no),
        FOREIGN KEY (dept_no) REFERENCES departments(dept_no)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dept_manager (
        emp_no INT NOT NULL,
        dept_no CHAR(4) NOT NULL,
        PRIMARY KEY (emp_no, dept_no),
        FOREIGN KEY (emp_no) REFERENCES employees(emp_no),
        FOREIGN KEY (dept_no) REFERENCES departments(dept_no)
    );
    """
]
# Execute table creation
for command in create_tables_sql:
    cursor.execute(command)

connection.commit()
print("All tables created successfully.")

# Step 2: Load Data into Tables
def load_csv_to_sqlite(file_path, table_name, conn):
    try:
        df = pd.read_csv(file_path)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Data from {file_path} loaded into {table_name}.")
    except FileNotFoundError:
        print(f"File not found: {file_path}. Skipping {table_name}.")
    except Exception as e:
        print(f"Error loading {file_path} into {table_name}: {e}")

# File paths
csv_files = {
    "departments": f"{directory}/departments.csv",
    "employees": f"{directory}/employees.csv",
    "salaries": f"{directory}/salaries.csv",
    "titles": f"{directory}/titles.csv",
    "dept_emp": f"{directory}/dept_emp.csv",
    "dept_manager": f"{directory}/dept_manager.csv"
}

# Load data into tables
for table_name, file_path in csv_files.items():
    load_csv_to_sqlite(file_path, table_name, connection)

# Step 3: Verify Table Data
def check_table_data(table_name, conn):
    try:
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        result = pd.read_sql(query, conn)
        print(f"Data preview for table: {table_name}")
        print(result)
    except Exception as e:
        print(f"Error checking data for {table_name}: {e}")

# Verify data in tables
for table_name in csv_files.keys():
    check_table_data(table_name, connection)

# Step 4: Data Analysis Queries
def run_query(query, conn):
    try:
        result = pd.read_sql(query, conn)
        print(result)
    except Exception as e:
        print(f"Error running query: {e}")

# Queries
queries = {
    "1. Employee details and salaries": """
        SELECT employees.emp_no, employees.last_name, employees.first_name, salaries.salary
        FROM employees
        JOIN salaries ON employees.emp_no = salaries.emp_no;
    """,
    "2. Employees hired in 1986": """
        SELECT first_name, last_name, hire_date
        FROM employees
        WHERE hire_date LIKE '1986%';
    """,
    "3. Managers and their departments": """
        SELECT dept_manager.dept_no, departments.dept_name, employees.emp_no, employees.last_name, employees.first_name
        FROM dept_manager
        JOIN departments ON dept_manager.dept_no = departments.dept_no
        JOIN employees ON dept_manager.emp_no = employees.emp_no;
    """,
    "4. Department details for employees": """
        SELECT dept_emp.dept_no, employees.emp_no, employees.last_name, employees.first_name, departments.dept_name
        FROM dept_emp
        JOIN employees ON dept_emp.emp_no = employees.emp_no
        JOIN departments ON dept_emp.dept_no = departments.dept_no;
    """,
    "5. Employees named Hercules B*": """
        SELECT first_name, last_name
        FROM employees
        WHERE first_name = 'Hercules' AND last_name LIKE 'B%';
    """,
    "6. Employees in Sales department": """
        SELECT employees.emp_no, employees.last_name, employees.first_name
        FROM employees
        JOIN dept_emp ON employees.emp_no = dept_emp.emp_no
        JOIN departments ON dept_emp.dept_no = departments.dept_no
        WHERE departments.dept_name = 'Sales';
    """,
    "7. Employees in Sales and Development": """
        SELECT employees.emp_no, employees.last_name, employees.first_name, departments.dept_name
        FROM employees
        JOIN dept_emp ON employees.emp_no = dept_emp.emp_no
        JOIN departments ON dept_emp.dept_no = departments.dept_no
        WHERE departments.dept_name IN ('Sales', 'Development');
    """,
    "8. Frequency of last names": """
        SELECT last_name, COUNT(last_name) AS name_count
        FROM employees
        GROUP BY last_name
        ORDER BY name_count DESC;
    """
}

# Execute and display each query
for description, query in queries.items():
    print(f"\n{description}")
    run_query(query, connection)

# Close the connection
connection.close()
print("Database connection closed.")


Directory exists. Files in the directory:
['EmployeeSQL', '.DS_Store']
All tables created successfully.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/departments.csv. Skipping departments.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/employees.csv. Skipping employees.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/salaries.csv. Skipping salaries.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/titles.csv. Skipping titles.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/dept_emp.csv. Skipping dept_emp.
File not found: /Users/sarahrwolberg/Desktop/UPENN_Data_Science_Bootcamp/Module_9/sql_challenge/dept_manager.csv. Skipping dept_manager.
Data preview for table: departments
  dept_no        dept_name
0    d001        Marketing
1    d002          F